# Bugzilla的Rest API调用示例

In [9]:
import datetime
import requests
import json

## 获取token

In [34]:
class Bugzilla:
    __bugzilla_url = 'http://devops.hansong-china.com/bugzilla/show_bug.cgi?id='
    __auth_url = 'http://devops.hansong-china.com/bugzilla/rest.cgi/login?login=steve.yang@hansong-china.com&password=pwd1013001'
    __auth_token = ''
    __member_list = ['steve.yang@hansong-china.com']
    
    def fetch_token(self):
        resp = requests.get(self.__auth_url)
        if resp.status_code != 200:
            print("Failed to get token(http got %d)" %(resp.status_code))
            self.__auth_token='BAD'
        else:
            print("Good")
            print(resp.text)
            jsdata = json.loads(resp.text)
            print(jsdata['token'])
            if 'token' in jsdata:
                self.__auth_token = jsdata['token']
                print("The token is %s" %(self.__auth_token))
        return 0
    
    
    def get_active_bug(self, who, last_time = None):
        active_bugs = []
        resolved_bugs = []
        
        search_url = 'http://devops.hansong-china.com/bugzilla/rest.cgi/bug?token=%s&assigned_to=%s' \
          %(self.__auth_token, who)
        resp = requests.get(search_url)
        if resp.status_code != 200:
            print("incorrect HTML data")
            return (active_bugs, resolved_bugs)
        
        bugs = json.loads(resp.text)['bugs']
        for it in bugs:
            if it['status'] != 'RESOLVED' and it['status'] != 'VERIFIED':
                item = {
                    'bugid': it['id'],
                    'project': it['product'],
                    'title': it['summary'],
                    'realname': it['assigned_to_detail']['real_name'],
                    'url': '%s%d' %(self.__bugzilla_url , it['id'])
                }
                
                active_bugs.append(item)
           
                
        # need time for the resolved bug cases..   
        search_url = 'http://devops.hansong-china.com/bugzilla/rest.cgi/bug?token=%s&assigned_to=%s&last_change_time=%s' \
          %(self.__auth_token, who, last_time)
        resp = requests.get(search_url)
        if resp.status_code != 200:
            print("incorrect HTML data")
            return (active_bugs, resolved_bugs)
        
        bugs = json.loads(resp.text)['bugs']
        for it in bugs:
            if it['status'] == 'RESOLVED' or it['status'] == 'VERIFIED':
                item = {
                    'bugid': it['id'],
                    'project': it['product'],
                    'title': it['summary'],
                    'realname': it['assigned_to_detail']['real_name'],
                    'url': '%s%d' %(self.__bugzilla_url , it['id'])
                }
                
                resolved_bugs.append(item)
           
        
        return (active_bugs, resolved_bugs)
    
    def fetch_bug(self, who, last_time):
        bug_list = []
        search_url = 'http://devops.hansong-china.com/bugzilla/rest.cgi/bug?token=%s&assigned_to=%s' %(self.__auth_token, who)
        resp = requests.get(search_url)
        if resp.status_code != 200:
            print("incorrect HTML data")
            return bug_list
        
        # parse these data
        bugs = json.loads(resp.text)['bugs']
        print("there are totally %d bugs for %s" %(len(bugs), who))
        
        search_url = 'http://devops.hansong-china.com/bugzilla/rest.cgi/bug?token=%s&assigned_to=%s&last_change_time=%s' \
          %(self.__auth_token, who, last_time)
        resp = requests.get(search_url)
        if resp.status_code != 200:
            print("incorrect HTML data")
            return bug_list
        
        # parse these data
        bugs = json.loads(resp.text)['bugs']
        print("After checking with time, there are totally %d bugs for %s" %(len(bugs), who))
        
        return bug_list
    
    def start(self):
        self.fetch_token()
        if self.__auth_token == 'BAD':
            print("Bad token, refuse to do this!")
            return -1
        
        for it in self.__member_list:
            self.fetch_bug(it, '2019-08-30T02:37:55Z')
        
        return 0
    
    def weekly_report(self):
        # Just fetching report data from DB!
        cur = datetime.datetime.now()
        return 0
    
    
    ## daily script would run below util
    def daily_report(self):
        cur = datetime.datetime.now()
        yesterday = cur - datetime.timedelta(days = 1)
        self.fetch_token()
        if self.__auth_token == 'BAD':
            print("Bad token, refuse to do this!")
            return -1
        
        for it in self.__member_list:
            (active_bugs, resolved_bugs) = self.get_active_bug(it, yesterday)
            print("%d active bugs found, %d bugs fixed" %(len(active_bugs), len(resolved_bugs)))
            for bugit in active_bugs:
                print("Bug-%s : %s" %(bugit['bugid'], bugit['title']))
            print("below are resolved bugs")
            for bugit in resolved_bugs:
                print("Bug-%s : %s" %(bugit['bugid'], bugit['title']))
        return 0
    
## Trigger the code...
bug = Bugzilla()
bug.daily_report()

Good
{"token":"1-vIXHkLU3GF","id":1}
1-vIXHkLU3GF
The token is 1-vIXHkLU3GF
28 active bugs found, 0 bugs fixed
Bug-21 : [Memo] - Try store the IoT cmds data to NoSQL instead of MySQL
Bug-22 : Change the timezone of Bugzilla
Bug-29 : [cloud] Need to separate the product env from the test env in Europe Region
Bug-45 : Set to microdot with Long wifi password(such as 27-digit) would be failed to setup the wifi connection.
Bug-46 : Provide an architecture design for the multi-region support feature
Bug-47 : Create libre cloud platform project for Vodafone
Bug-58 : Deploy libre cloud project
Bug-60 : Test env deployment is failed in EU region
Bug-61 : gitlab's admin email is setting in an incorrect way
Bug-63 : Need to support the SSL certificate as Alex Skill requires this.
Bug-65 : AWS下两个账号需要关联统一支付费用账单
Bug-115 : multi-region-cvm region select
Bug-116 : multi-region-IoT endpoint
Bug-117 : multi-region-policy mgr
Bug-118 : User hama url to do alexa account link
Bug-119 : multi-region-account

0

In [ ]:
## 格式化日期的示例

In [13]:
bug = Bugzilla()
bug.start()

Good
{"token":"1-L1oiwRG8O8","id":1}


0

## 周的操作
### .weekday()返回0对应周一，$\cdots$, 6对应周日

In [10]:
today = datetime.datetime.now()
dow = today.weekday()
print(dow)

from_date = today + datetime.timedelta(days = -dow)
to_date = from_date + datetime.timedelta(days = 5)
print(from_date)
print(to_date)

0
2019-08-26 03:56:30.624514
2019-08-31 03:56:30.624514
